In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)


In [2]:
modern_topic_df = pd.read_csv('/vast/amr10211/SCDB_2022_01_caseCentered_Citation.csv', encoding='latin-1')

In [3]:
modern_topic_df = modern_topic_df[['caseId', 'issueArea']]

In [4]:
legacy_topic_df = pd.read_csv('/vast/amr10211/SCDB_Legacy_07_caseCentered_Citation.csv', encoding='latin-1')

In [5]:
legacy_topic_df = legacy_topic_df[['caseId', 'issueArea']]

In [6]:
topic_df = pd.concat((legacy_topic_df,modern_topic_df),axis=0)

In [7]:
cluster_df = pd.read_csv('/vast/amr10211/opinions-cluster-data-sc.csv')

In [8]:
cluster_df['date_filed'] = pd.to_datetime(cluster_df['date_filed'], errors='coerce')
cluster_df = cluster_df[cluster_df['date_filed'].dt.year >= 1930]

In [9]:
cluster_df = cluster_df[['id','scdb_id']]

In [10]:
merged_df = cluster_df.merge(topic_df, how = 'left', left_on='scdb_id', right_on= 'caseId')

In [11]:
merged_df.count()

id           11496
scdb_id      11496
caseId       11496
issueArea    11469
dtype: int64

In [12]:
merged_df['issueArea'].value_counts()

8.0     2749
1.0     2216
9.0     1583
2.0     1574
12.0     750
3.0      718
10.0     501
7.0      472
4.0      414
11.0     123
6.0      120
5.0      118
14.0      92
13.0      39
Name: issueArea, dtype: int64

In [13]:
merged_df

,id,scdb_id,caseId,issueArea
0,105739,1957-136,1957-136,8.0
1,105740,1957-137,1957-137,7.0
2,105749,1957-148,1957-148,1.0
3,105750,1957-149,1957-149,1.0
4,105760,1957-154,1957-154,1.0
...,...,...,...,...
11491,102457,1934-063,1934-063,8.0
11492,102408,1934-165,1934-165,2.0
11493,102409,1934-167,1934-167,3.0
11494,102411,1934-169,1934-169,4.0


In [11]:
sum_df = pd.read_csv('summarized-opinions-data-sc.csv')
train_df = pd.read_csv('/vast/amr10211/sc-train.csv')
val_df = pd.read_csv('/vast/amr10211/sc-val.csv')
test_df = pd.read_csv('/vast/amr10211/sc-test.csv')

sum_df = sum_df.merge(merged_df, how='left', left_on='cluster_id', right_on='id')
sum_df = sum_df.drop(['Unnamed: 0','id_y', 'scdb_id', 'caseId'], axis=1)
sum_df = sum_df.rename(columns={'id_x': 'id', 'issueArea': 'issue_area'})
sum_df = sum_df.dropna(subset=['issue_area'])

train_df = train_df.merge(merged_df, how='left', left_on='cluster_id', right_on='id')
train_df = train_df.drop(['Unnamed: 0','id_y', 'scdb_id', 'caseId'], axis=1)
train_df = train_df.rename(columns={'id_x':'id', 'issueArea': 'issue_area'})
train_df = train_df.dropna(subset=['issue_area'])

val_df = val_df.merge(merged_df, how='left', left_on='cluster_id', right_on='id')
val_df = val_df.drop(['Unnamed: 0','id_y', 'scdb_id', 'caseId'], axis=1)
val_df = val_df.rename(columns={'id_x':'id', 'issueArea': 'issue_area'})
val_df = val_df.dropna(subset=['issue_area'])

test_df = test_df.merge(merged_df, how='left', left_on='cluster_id', right_on='id')
test_df = test_df.drop(['Unnamed: 0','id_y', 'scdb_id', 'caseId'], axis=1)
test_df = test_df.rename(columns={'id_x':'id', 'issueArea': 'issue_area'})
test_df = test_df.dropna(subset=['issue_area'])


In [12]:
sum_df.to_csv('summarized-opinions-data-sc-topic.csv')
train_df.to_csv('/vast/amr10211/sc-train-topic.csv')
val_df.to_csv('/vast/amr10211/sc-val-topic.csv')
test_df.to_csv('/vast/amr10211/sc-test-topic.csv')

In [16]:
sum_df.head()

,id,cluster_id,type,decision_text,date_filed,scdb_decision_direction,issue_area
0,103341,103341,010combined,"In this paper, Justice Black defends the provi...",1940-05-20,2.0,8.0
1,9419101,103344,020lead,"In this case, the foreign corporations brought...",1940-04-22,2.0,8.0
2,9419102,103344,040dissent,"In this case, Justice Roberts disagrees with t...",1940-04-22,1.0,8.0
3,103348,103348,010combined,"In this case, the Court argues that a local la...",1940-05-20,2.0,3.0
4,103350,103350,010combined,"In this case, Reed argues that the railroads B...",1940-04-29,1.0,8.0
